# Autoencoder Training Notebook

This notebook trains the autoencoder and computes the dynamic threshold.

In [ ]:
info ="""

Author: Annam.ai IIT Ropar
Team Name: SoilClassifiers
Team Members: Caleb Chandrasekar, Sarvesh Chandran, Swaraj Bhattacharjee, Karan Singh, Saatvik Tyagi
Leaderboard Rank: 120

"""

In [ ]:
# training.py

import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm

# ✅ Environment
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Configuration
IMG_SIZE   = 224
BATCH_SIZE = 32
EPOCHS     = 30
THRESHILE_PERCENTILE = 90  # percentile for dynamic threshold

# ✅ Custom Dataset
class SoilDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder      = folder
        self.image_files = [f for f in os.listdir(folder)
                            if f.lower().endswith(('.png','jpg','jpeg'))]
        self.transform   = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder, img_name)
        img      = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_name

# ✅ Transforms
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ✅ Autoencoder definition
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,  32, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(64,128, 3, stride=2, padding=1), nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64,  32, 3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(32,   3, 3, stride=2, padding=1, output_padding=1), nn.Sigmoid(),
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

# ✅ Prepare data loader
train_ds = SoilDataset(
    "/kaggle/input/soil-classification-part-2/soil_competition-2025/train",
    transform=train_transforms
)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

# ✅ Model, loss & optimizer
model     = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ✅ Training loop
for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    for imgs, _ in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        imgs = imgs.to(device)
        recon = model(imgs)
        loss  = criterion(recon, imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch:2d}/{EPOCHS} — train loss: {avg_loss:.6f}")

# ✅ Compute dynamic threshold (90th percentile of train losses)
model.eval()
train_losses = []
with torch.no_grad():
    for imgs, _ in train_loader:
        imgs = imgs.to(device)
        recon = model(imgs)
        train_losses.append(criterion(recon, imgs).item())

threshold = np.percentile(train_losses, THRESHILE_PERCENTILE)
print(f"Dynamic threshold ({THRESHILE_PERCENTILE}th percentile): {threshold:.6f}")

# ✅ Save model & threshold
torch.save(model.state_dict(), "autoencoder.pth")
np.save("threshold.npy", np.array(threshold))
print("✅ Saved: autoencoder.pth + threshold.npy")
